In [ ]:
from IPython.display import display_html
display_cols = lambda dfs,titles=[],cols=100:display_html("".join(f'<table><thead>{"".join(f"<th>{title}</th>" for title in titles_slice)}</thead><tbody>{"".join("""<td style="vertical-align: top;">"""+f"{df.to_html()}</td>" for df in dfs_slice)}</tbody></table>' for titles_slice,dfs_slice in [(titles[:len(dfs)][i:i+cols],dfs[i:i+cols]) for i in range(0,len(dfs),cols)]),raw=True)

In [ ]:
### Q9
import pandas as pd
import numpy as np
t6 = """Age, Education ,Job Suitability
26 ,Master ,High
25 ,High School ,Very Low
34 ,High School, Low
51 ,High School, High
45 ,High School ,Very Low
46 ,Master, High
36 ,Master, High
38 ,Bachelor, Low
27, Master, Very High"""

t7 = """Age, Education, Job Suitability
[18-35], University, High
[18-35], High School, Very Low
[18-35], High School, Low
[36-65], High School, High
[36-65], High School, Very Low
[36-65], University, High
[36-65], University, High
[36-65], University, Low
[18-35], University, Very High"""

t8_1 = """Age, Education, Group ID
26 ,Master ,1
25 ,High School ,2
34 ,High School ,2
51 ,High School ,3
45 ,High School ,3
46 ,Master, 4
36 ,Master ,4
38 ,Bachelor, 4
27 ,Master, 1"""

t8_2 = """Group ID ,Job Suitability ,Count
1, High, 1
1 ,Very High ,1
2, Very Low ,1
2 ,Low, 1
3 ,Very Low, 1
3 ,High, 1
4 ,Low ,1
4 ,High ,2"""


df_t6 = t6.splitlines()
df_t6 = [[x.strip() for x in y.split(",")] for y in df_t6]
df_t6 = pd.DataFrame(df_t6[1:],columns=df_t6[0])

df_t7 = t7.splitlines()
df_t7 = [[x.strip() for x in y.split(",")] for y in df_t7]
df_t7 = pd.DataFrame(df_t7[1:],columns=df_t7[0])

df_t8_1 = t8_1.splitlines()
df_t8_1 = [[x.strip() for x in y.split(",")] for y in df_t8_1]
df_t8_1 = pd.DataFrame(df_t8_1[1:],columns=df_t8_1[0])

df_t8_2 = t8_2.splitlines()
df_t8_2 = [[x.strip() for x in y.split(",")] for y in df_t8_2]
df_t8_2 = pd.DataFrame(df_t8_2[1:],columns=df_t8_2[0])

display_cols([df_t6,df_t7],["Table 6","Table 7"])
display_cols([df_t8_1,df_t8_2],["Table 8.1","Table 8.2"])

In [ ]:
## (a)
import base64
print(base64.b64decode(
    b'Ck51bWVyaWMgYXR0cmlidXRlIEFnZSB3YXMgZ3JvdXBlZCBpbnRvIHJhbmdlcy'
    b'4KQ2F0ZWdvcmljYWwgYXR0cmlidXRlIEVkdWNhdGlvbiB3YXMgYWJzdHJhY3Rl'
    b'ZCwgYnkgY29tYmluaW5nIEJhY2hlbG9yIGFuZCBNYXN0ZXIgaW50byBVbml2ZX'
    b'JzaXR5Cg=='
    ).decode()
)

In [ ]:
## (b)
# (1)
# distinct-l-diversity is the minimum number of distinct values for all sensitive attributes in the same equivalency class,
# where the equivalency class is a unique combination of (quasi)-identifying attributes
cell_df = df_t7.copy()
quasi_identifiers = ["Age","Education"]
sensitive_attributes = ["Job Suitability"]
eq_classes = list()

def combinations(q_ids,df,prev=[]):
    if len(q_ids)==1:
        for unique in df[q_ids[0]].unique():
            yield prev + [unique]
    else:
        for unique in df[q_ids[0]].unique():
            yield from combinations(q_ids[1:],df,prev=prev+[unique])

for combination in combinations(quasi_identifiers,cell_df):
    filter = pd.Series([True]*len(cell_df),index=cell_df.index)
    for qid,qid_val in zip(quasi_identifiers,combination):
        filter &= (cell_df[qid]==qid_val)
    eq_classes.append(cell_df[filter])


def l_diversity(df_eq,sens_attr:list):
    ldiversity = len(df_eq[sens_attr[0]].unique())
    for attr in sens_attr[1:]:
        attr_ldiv = len(df_eq[attr].unique())
        ldiversity = min(ldiversity,attr_ldiv)
    return ldiversity

display_cols(
    eq_classes,
    titles=[f"L-Diversity: {l_diversity(df_eq,sensitive_attributes)}" for df_eq in eq_classes],
    cols=2
    )

# (2) Can't find a definition for recursive-(c,l)-diversity
# But for fun here is Entropy l-diversity
def entropy(df_eq,sens_attr:list):
    attr_entropy = 0
    for attr in sens_attr:
        attr_entropy += -sum(df_eq[attr].value_counts(normalize=True)*np.log2(df_eq[attr].value_counts(normalize=True)))
    return attr_entropy


display_cols(
    eq_classes,
    titles=[f"Entropy L-Diversity: {np.ceil(2**entropy(df_eq,sensitive_attributes))}" for df_eq in eq_classes],
    cols=2
)

In [ ]:
## (c)
# (1)
query = [("Age","51"),("Job Suitability","High")]
query_df = df_t6.copy()
filter = pd.Series([True]*len(query_df),index=query_df.index)
for qid,qid_val in query:
    filter &= (query_df[qid]==qid_val)
query_df = query_df[filter]
display_cols([query_df],[f"Query produced {len(query_df)} rows"])

In [ ]:
# (2)
query = [("Age","51"),("Job Suitability","High")]
query_df = df_t7.copy()
filter = pd.Series([True]*len(query_df),index=query_df.index)
for qid,qid_val in query:
    if qid=="Age":
        qid_val = int(qid_val)
        filter &= query_df[qid].apply(lambda x: int(x.split("-")[0][1:])<=qid_val and int(x.split("-")[1][:-1])>=qid_val)
    else:
        filter &= (query_df[qid]==qid_val)
query_df = query_df[filter]
display_cols([query_df],[f"Query produced {len(query_df)} rows"])

# Here we don't have a concrete answer because we can't guarantee there is a 51 year old in the range [36-65] with job suitability "High"
# But we do know that there are 3 people who might match the query so we can compute an excpected count by dividing the count
# of the matching rows by the age range, if we assume uniform distribution of ages
age_range = 65-36
expected_count = len(query_df)/age_range
print(f"Expected count: {expected_count}")

In [ ]:
# (3)
# One guy in group 3 is 51 (from table 8.1) and one guy in group 3 has high job suitability (from table 8.2)
# But we can't be sure that it's the same guy so we can't answer the question accurately
# I don't know if this is the correct answer but the chance of the 51 year old in group 3 having high job suitability is
# 0.5 because there are 2 people in group 3 and one of them has high job suitability
# So I guess the expected count is 0.5

In [ ]:
## (d)
# (1) No.
# (2) Accuracy is sum of true positives and true negatives divided by the total number of samples
# So 4 correctly predicted as positive and 3 correctly predicted as negative
accuracy = (4+3)/9
print(f"Accuracy: {accuracy:.2f}")
# (3) Discrimination is the number of unprotected positives divided by the number of 
# unprotected - the number of protected positives divided by the number of protected
# So 2 unprotected positives (4 unprotected in total) and 3 protected positives (5 protected in total)
discrimination = (2/4) - (3/5)
print(f"Discrimination: {discrimination:.2f}")